# Análisis y Formación de Equipos Balanceados

Este notebook contiene el código completo para analizar datos de estudiantes y proponer una distribución óptima en dos equipos balanceados, considerando habilidades técnicas y comunicativas.

## 1. Configuración del Entorno

Importamos las librerías necesarias para el análisis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configurar estilo de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configurar visualización de todas las columnas
pd.set_option('display.max_columns', None)

## 2. Carga y Exploración de Datos

Cargamos los datos desde el archivo Excel y exploramos su estructura.

In [ ]:
# Cargar el archivo Excel - Ajusta la ruta según donde tengas el archivo
file_path = 'TeamBuilding_Famaf_2025.xlsx'

# Leer la primera hoja del archivo
df = pd.read_excel(file_path, sheet_name=0)

# Mostrar información básica del dataframe
print('Información del DataFrame:')
print(df.info())
print('\nPrimeras filas del DataFrame:')
display(df.head())
print('\nEstadísticas descriptivas:')
display(df.describe())

## 3. Limpieza y Preparación de Datos

Limpiamos los datos y preparamos el DataFrame para el análisis.

In [ ]:
# Limpiar los nombres de las columnas para facilitar su uso
df.columns = [col.strip() for col in df.columns]

# Renombrar columnas para facilitar su uso
df = df.rename(columns={
    'Uso Github?': 'Github',
    'Habilidades c/Herram. de Presentacion? (Ej. Canva, Power Point, Goggle Slides, etc)': 'Presentacion',
    'otras...(Ej. VSCode, MLFlow, Streamlit, etc)': 'Herramientas_Tecnicas'
})

# Filtrar filas con datos completos de habilidades
df_complete = df.dropna(subset=['Github', 'Presentacion', 'Herramientas_Tecnicas', 'Nombre'])

# Mostrar información de los estudiantes con datos completos
print('Estudiantes con datos completos de habilidades:')
display(df_complete[['Nombre', 'Github', 'Presentacion', 'Herramientas_Tecnicas', 'Experiencia previa en proyectos similares (Sí / No)']])

## 4. Análisis Estadístico de Habilidades

Analizamos la distribución de habilidades entre los estudiantes.

In [ ]:
# Análisis estadístico de las habilidades
print('Estadísticas de habilidades:')
display(df_complete[['Github', 'Presentacion', 'Herramientas_Tecnicas']].describe())

# Identificar niveles de habilidad por estudiante
print('\nNiveles de habilidad por estudiante:')
for idx, row in df_complete.iterrows():
    print(f"Estudiante: {row['Nombre']}:")
    print(f"  - Github: {row['Github']}")
    print(f"  - Presentación: {row['Presentacion']}")
    print(f"  - Herramientas Técnicas: {row['Herramientas_Tecnicas']}")
    print(f"  - Experiencia previa: {row['Experiencia previa en proyectos similares (Sí / No)']}")
    print()

## 5. Visualización de Habilidades

Generamos visualizaciones para entender mejor la distribución de habilidades.

In [ ]:
# 1. Gráfico de barras para comparar habilidades entre estudiantes
plt.figure(figsize=(14, 8))
df_skills = df_complete[['Nombre', 'Github', 'Presentacion', 'Herramientas_Tecnicas']]
df_skills_melted = pd.melt(df_skills, id_vars=['Nombre'], 
                          value_vars=['Github', 'Presentacion', 'Herramientas_Tecnicas'],
                          var_name='Habilidad', value_name='Nivel')

# Crear gráfico de barras agrupadas
sns.barplot(x='Nombre', y='Nivel', hue='Habilidad', data=df_skills_melted)
plt.title('Comparación de Habilidades por Estudiante', fontsize=16)
plt.xlabel('Estudiante', fontsize=14)
plt.ylabel('Nivel de Habilidad (1-5)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Tipo de Habilidad')
plt.tight_layout()
plt.show()

In [ ]:
# 2. Gráfico de radar para cada estudiante
def create_radar_chart(student_data, student_name):
    # Preparar datos para el gráfico de radar
    categories = ['Github', 'Presentacion', 'Herramientas_Tecnicas']
    values = student_data[categories].values.flatten().tolist()
    
    # Número de variables
    N = len(categories)
    
    # Ángulos para cada eje
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Cerrar el polígono
    
    # Valores para cada eje
    values += values[:1]  # Cerrar el polígono
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    
    # Dibujar polígono
    ax.plot(angles, values, linewidth=2, linestyle='solid')
    ax.fill(angles, values, alpha=0.25)
    
    # Etiquetas
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    
    # Configurar límites del eje y
    ax.set_ylim(0, 5)
    
    # Título
    plt.title(f'Perfil de Habilidades: {student_name}', size=15, y=1.1)
    
    plt.tight_layout()
    plt.show()

# Generar un gráfico de radar para cada estudiante
for idx, row in df_complete.iterrows():
    student_name = row['Nombre']
    create_radar_chart(row, student_name)

In [ ]:
# 3. Gráfico de distribución de habilidades en todo el grupo
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_complete[['Github', 'Presentacion', 'Herramientas_Tecnicas']])
plt.title('Distribución de Habilidades en el Grupo', fontsize=16)
plt.ylabel('Nivel de Habilidad (1-5)', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# 4. Mapa de calor de habilidades
plt.figure(figsize=(12, 8))
heatmap_data = df_complete.set_index('Nombre')[['Github', 'Presentacion', 'Herramientas_Tecnicas']]
sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu', linewidths=.5, fmt='.1f')
plt.title('Mapa de Calor de Habilidades por Estudiante', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# 5. Gráfico de dispersión para visualizar relaciones entre habilidades
plt.figure(figsize=(10, 8))
sns.scatterplot(x='Github', y='Presentacion', size='Herramientas_Tecnicas', 
                sizes=(100, 500), data=df_complete, alpha=0.7)

# Añadir nombres de estudiantes como etiquetas
for idx, row in df_complete.iterrows():
    plt.annotate(row['Nombre'], (row['Github'], row['Presentacion']), 
                 xytext=(5, 5), textcoords='offset points')

plt.title('Relación entre Habilidades Técnicas y de Presentación', fontsize=16)
plt.xlabel('Nivel de Github (1-5)', fontsize=14)
plt.ylabel('Nivel de Presentación (1-5)', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## 6. Algoritmo para Proponer Equipos Balanceados

Implementamos el algoritmo para distribuir a los estudiantes en equipos balanceados.

In [ ]:
# Calcular puntaje total por estudiante
df_complete['Total'] = df_complete['Github'] + df_complete['Presentacion'] + df_complete['Herramientas_Tecnicas']

# Crear una función para proponer equipos balanceados
def propose_balanced_teams(students_df):
    # Crear una copia para no modificar el original
    df_copy = students_df.copy()
    
    # Calcular puntaje técnico (Github + Herramientas_Tecnicas)
    df_copy['Tech_Score'] = df_copy['Github'] + df_copy['Herramientas_Tecnicas']
    df_copy['Comm_Score'] = df_copy['Presentacion']
    
    # Ordenar por puntaje técnico de mayor a menor
    tech_sorted = df_copy.sort_values('Tech_Score', ascending=False).reset_index(drop=True)
    
    # Inicializar equipos
    team1 = []
    team2 = []
    
    # Distribuir estudiantes alternando entre equipos (serpenteo)
    for i in range(len(tech_sorted)):
        if i % 2 == 0:
            team1.append(tech_sorted.iloc[i])
        else:
            team2.append(tech_sorted.iloc[i])
    
    # Convertir a DataFrames
    team1_df = pd.DataFrame(team1).reset_index(drop=True)
    team2_df = pd.DataFrame(team2).reset_index(drop=True)
    
    return team1_df, team2_df

# Proponer equipos
team1, team2 = propose_balanced_teams(df_complete)

# Mostrar los equipos propuestos
print('EQUIPO 1:')
display(team1[['Nombre', 'Github', 'Presentacion', 'Herramientas_Tecnicas', 'Tech_Score', 'Comm_Score', 'Total', 'Experiencia previa en proyectos similares (Sí / No)']])

print('\nEQUIPO 2:')
display(team2[['Nombre', 'Github', 'Presentacion', 'Herramientas_Tecnicas', 'Tech_Score', 'Comm_Score', 'Total', 'Experiencia previa en proyectos similares (Sí / No)']])

## 7. Validación del Balance de Equipos

Calculamos estadísticas para validar el balance entre los equipos.

In [ ]:
# Calcular estadísticas de los equipos
def calculate_team_stats(team_df):
    stats = {
        'Github_avg': team_df['Github'].mean(),
        'Presentacion_avg': team_df['Presentacion'].mean(),
        'Herramientas_Tecnicas_avg': team_df['Herramientas_Tecnicas'].mean(),
        'Tech_Score_avg': team_df['Tech_Score'].mean(),
        'Comm_Score_avg': team_df['Comm_Score'].mean(),
        'Total_avg': team_df['Total'].mean()
    }
    return stats

team1_stats = calculate_team_stats(team1)
team2_stats = calculate_team_stats(team2)

# Mostrar estadísticas de los equipos
print('Estadísticas Equipo 1:')
for key, value in team1_stats.items():
    print(f'{key}: {value:.2f}')

print('\nEstadísticas Equipo 2:')
for key, value in team2_stats.items():
    print(f'{key}: {value:.2f}')

# Comparar estadísticas entre equipos
print('\nCOMPARACIÓN DE EQUIPOS:')
for key in team1_stats.keys():
    diff = abs(team1_stats[key] - team2_stats[key])
    print(f'Diferencia en {key}: {diff:.2f}')

In [ ]:
# Visualizar comparación de equipos
# Preparar datos para visualización
team_comparison = pd.DataFrame({
    'Habilidad': ['Github', 'Presentacion', 'Herramientas_Tecnicas', 'Tech_Score', 'Comm_Score', 'Total'],
    'Equipo 1': [team1_stats['Github_avg'], team1_stats['Presentacion_avg'], 
                team1_stats['Herramientas_Tecnicas_avg'], team1_stats['Tech_Score_avg'],
                team1_stats['Comm_Score_avg'], team1_stats['Total_avg']],
    'Equipo 2': [team2_stats['Github_avg'], team2_stats['Presentacion_avg'], 
                team2_stats['Herramientas_Tecnicas_avg'], team2_stats['Tech_Score_avg'],
                team2_stats['Comm_Score_avg'], team2_stats['Total_avg']]
})

# Convertir a formato largo para seaborn
team_comparison_melted = pd.melt(team_comparison, id_vars=['Habilidad'], 
                                value_vars=['Equipo 1', 'Equipo 2'],
                                var_name='Equipo', value_name='Promedio')

# Crear gráfico de barras comparativas
plt.figure(figsize=(12, 8))
sns.barplot(x='Habilidad', y='Promedio', hue='Equipo', data=team_comparison_melted)
plt.title('Comparación de Habilidades entre Equipos', fontsize=16)
plt.xlabel('Tipo de Habilidad', fontsize=14)
plt.ylabel('Promedio', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Generar un gráfico de radar para comparar los equipos
def create_team_radar_chart():
    # Preparar datos para el gráfico de radar
    categories = ['Github', 'Presentacion', 'Herramientas_Tecnicas']
    
    team1_values = [team1[cat].mean() for cat in categories]
    team2_values = [team2[cat].mean() for cat in categories]
    
    # Número de variables
    N = len(categories)
    
    # Ángulos para cada eje
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    
    # Cerrar el polígono repitiendo el primer valor
    team1_values += team1_values[:1]
    team2_values += team2_values[:1]
    angles += angles[:1]
    
    # Crear figura
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    # Dibujar polígonos
    ax.plot(angles, team1_values, linewidth=2, linestyle='solid', label='Equipo 1')
    ax.fill(angles, team1_values, alpha=0.25)
    
    ax.plot(angles, team2_values, linewidth=2, linestyle='solid', label='Equipo 2')
    ax.fill(angles, team2_values, alpha=0.25)
    
    # Etiquetas
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    
    # Configurar límites del eje y
    ax.set_ylim(0, 5)
    
    # Título y leyenda
    plt.title('Comparación de Habilidades entre Equipos', size=15, y=1.1)
    plt.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Generar gráfico de radar para comparación de equipos
create_team_radar_chart()

## 8. Análisis de Fortalezas y Áreas de Mejora

Identificamos las fortalezas y áreas de mejora de cada equipo.

In [ ]:
# Análisis de experiencia previa en cada equipo
print('Análisis de experiencia previa:')
print('Equipo 1:')
display(team1[['Nombre', 'Experiencia previa en proyectos similares (Sí / No)']])
print('\nEquipo 2:')
display(team2[['Nombre', 'Experiencia previa en proyectos similares (Sí / No)']])

## 9. Conclusiones y Recomendaciones

Presentamos las conclusiones y recomendaciones basadas en el análisis.

### Fortalezas del Equipo 1

- Cuenta con Manuel Lopez Werlen, quien tiene alto nivel tanto en Github (5) como en Herramientas Técnicas (4).
- Tiene a Ayelen Margarita Bertorello, la única estudiante con experiencia previa en proyectos similares.
- Buen nivel promedio en habilidades de presentación (3.5).

### Áreas de Mejora del Equipo 1

- Tres de los cuatro integrantes tienen nivel bajo (1) en Github, dependiendo principalmente de un solo miembro para esta habilidad.

### Fortalezas del Equipo 2

- Cuenta con Julian Rametta, quien tiene un perfil equilibrado en todas las habilidades (Github: 5, Presentación: 3, Herramientas Técnicas: 3).
- Excelente nivel en habilidades de presentación (promedio 3.75), con dos miembros en nivel 5.

### Áreas de Mejora del Equipo 2

- Ningún integrante tiene experiencia previa en proyectos similares.
- Tres de los cuatro integrantes tienen nivel bajo (1) en Github, dependiendo principalmente de un solo miembro para esta habilidad.

### Recomendaciones

1. **Capacitación en Github**: Dado que la mayoría de los estudiantes tienen un nivel bajo en Github (6 de 8 con nivel 1), se recomienda realizar una capacitación inicial en esta herramienta para todos los equipos.

2. **Mentorías cruzadas**: Aprovechar a los estudiantes con niveles altos en ciertas habilidades para que compartan conocimientos con sus compañeros:
   - Manuel Lopez Werlen y Julian Rametta pueden liderar sesiones sobre Github y herramientas técnicas.
   - Mirna Hughes y Eloy Moyano pueden compartir sus conocimientos sobre presentaciones efectivas.
   - Ayelen Margarita Bertorello puede compartir su experiencia previa en proyectos similares con ambos equipos.

3. **Seguimiento periódico**: Implementar revisiones periódicas del avance de los equipos para identificar posibles desequilibrios o dificultades que puedan surgir durante el desarrollo del proyecto.

## 10. Conclusión Final

La distribución propuesta logra un equilibrio notable entre ambos equipos en términos de habilidades técnicas y comunicativas, con diferencias mínimas en los promedios de cada habilidad. Esta configuración permite que cada equipo tenga acceso a un conjunto diverso de habilidades, facilitando el desarrollo integral del proyecto de ciencia de datos end-to-end.

Aunque existen algunas brechas, particularmente en la experiencia previa y en el dominio de Github, las recomendaciones propuestas buscan mitigar estas limitaciones y promover un ambiente de aprendizaje colaborativo entre los estudiantes.